In [1]:
import os
 
from sqlalchemy import Column, DateTime, String, Integer, ForeignKey, func, create_engine, Sequence
from sqlalchemy.orm import relationship, backref, sessionmaker
from sqlalchemy.ext.declarative import declarative_base

import pandas as pd
import requests

In [3]:
engine = create_engine('sqlite:///:memory:')

In [4]:
Base = declarative_base()


class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
#     user_id = Column(Integer, ForeignKey('users.id'))
    
    user = relationship("User", back_populates="addresses")
    
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, Sequence('user_id_seq'), primary_key=True)
    name = Column(String(50))
    fullname = Column(String(50))
    nickname = Column(String(50))
    address_id = Column(Integer, ForeignKey('addresses.id'))

    addresses = relationship("Address", order_by=Address.id, back_populates="user")

    


Session = sessionmaker(bind=engine)
Base.metadata.create_all(bind=engine)

session = Session()
session.commit()
session.close()

In [5]:
session = Session()

john = User(name='john', fullname='john john', nickname='asfafgsgda')
jack = User(name='jack', fullname='Jack Bean', nickname='gjffdd')

session.add_all([jack, john])
session.commit()

In [6]:
ad = Address()
ad.email_address = 'bla@bla.com'
session.add(ad)

ad.user = [jack, john]

session.commit()

In [7]:
pd.read_sql_table('users', engine)

,id,name,fullname,nickname,address_id
0,1,jack,Jack Bean,gjffdd,1
1,2,john,john john,asfafgsgda,1


In [8]:
pd.read_sql_table('addresses', engine)

,id,email_address
0,1,bla@bla.com
